In [23]:
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
import astropy.units as u
import numpy as np
import Quaternion
from Quaternion import Quat
rng = np.random.default_rng(1337)

# 100_000 times randomly distributed over 12 hours
t = Time('2020-01-01T20:00:00') + rng.uniform(0, 1, 10_000) * u.hour

location = location = EarthLocation(
    lon=-17.89 * u.deg, lat=28.76 * u.deg, height=2200 * u.m
)

quat = Quat(q=Quaternion.normalize((1,20,30,0)))
coord = SkyCoord(*quat.equatorial[:2], frame="icrs", unit="degree")

# target horizontal coordinate frame
altaz = AltAz(obstime=t, location=location)

In [25]:
altaz

<AltAz Frame (obstime=['2020-01-01T20:52:41.167' '2020-01-01T20:11:07.901'
 '2020-01-01T20:55:15.242' ... '2020-01-01T20:23:24.606'
 '2020-01-01T20:35:26.593' '2020-01-01T20:30:37.996'], location=(5326958.3926938, -1719534.01587917, 3051667.79174796) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron)>

In [53]:
# real to pixel
def baseline_sep_to_fov(d_baseline, sensor_size=(30*10**-3, 20*10*-3), focal_l=0.2):
    """
    return Cartesian field of view in (h, w)
    """
    return np.reshape(sensor_size*d_baseline/focal_l, (1,len(sensor_size)))

def real_sep_to_camera(sep, fov, sensor_size_pixel=(600, 400)):
    """
    convert led separation to camera scale.
    """
    return sep*sensor_size_pixel/fov

def led_pos_camera(led_sep_camera):
    """
    get led positions as appeared in camera
    in camera pixels, but not quantised to pixel integers
    """
    return np.array(((1/2.0, 0), (-1/2.0, 0)))*led_sep_camera

def pixelate(x_camera):
    """
    for now just np.round, may need more precise model of pixel quantisation
    """
    return np.round(x_camera)

fov = baseline_sep_to_fov(np.array(30))
led_sep_camera = real_sep_to_camera(np.array(0.8), fov)
pixelate(led_pos_pixel(led_sep_pixel))

array([[ 53.,  -0.],
       [-53.,  -0.]])

In [47]:
# pixel to real

def pixel_sep_to_real(sep_pixel, fov, sensor_size_pixel=(600, 400)):
    """
    convert led separation in camera pixels to meters.
    """
    if len(sep_pixel.shape) == 2:
        return sep_pixel*fov/np.reshape(np.array(sensor_size_pixel), (1,2))
    elif len(sep_pixel.shape) == 1:
        return sep_pixel*fov/sensor_size_pixel[0]
    else:
        raise TypeError(f'sep_pixel of {sep_pixel.shape} is only allowed to have shape (n, 2) or (n, 1)')

def led_pos_pixel_to_offset(pos_pixel):
    """
    assume chief and deputies are aligned in the star direction
    return offset in pixels of 2d, scale in pixels between 2 leds, and angle in radian
    """
    led1 = pos_pixel[1]
    led2 = pos_pixel[2]
    sep = np.linalg.norm(led1-led2)
    angle = np.arctan2(*(led1-led2)[::-1])
    mid = (led1+led2)/2
    return (mid, sep, angle)

def offset_by_ideal(offset, ideal_pos):
    pass
    return (yz_offset, x_offset, angle)

array([1, 2])